# Frequent Itemset Mining

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [99]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('Online Retail.xlsx')
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [100]:
df2 = pd.read_excel('online_retail_II.xlsx')
df2 = df2.rename(
    columns={"Invoice": "InvoiceNo", "Price": "UnitPrice", "Customer ID": "CustomerID"}
)
df2.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [134]:
# df2 has different column name!
df2.head()
frames = [df, df2]
data = pd.concat(frames)
data = data[data['InvoiceNo'].notnull()]
data = data[data['Quantity'] > 0]
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data.head()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [135]:
data = data[data['Country'] == 'United Kingdom']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Question 1.3: What are the most popular 5 items?

In [136]:
data.groupby(["StockCode", "Description"])["Quantity"].sum().sort_values(
    ascending=False
).head(5)

StockCode  Description                       
84077      WORLD WAR 2 GLIDERS ASSTD DESIGNS     101464
85123A     WHITE HANGING HEART T-LIGHT HOLDER     88415
23843      PAPER CRAFT , LITTLE BIRDIE            80995
23166      MEDIUM CERAMIC TOP STORAGE JAR         77036
84879      ASSORTED COLOUR BIRD ORNAMENT          76021
Name: Quantity, dtype: int64

Question 1.4: Filter down the data to include transaction that contain the top 20 items

In [137]:
# get the codes of the top 20 most sold items
top_20_item_codes = (
    data.groupby("StockCode")["Quantity"].sum().sort_values(ascending=False).head(20)
).index.to_list()

# transactions that contain the top 20 items
top_20_transactions = (
    data[data["StockCode"].isin(top_20_item_codes)]["InvoiceNo"].unique().tolist()
)

data = data.where(data["InvoiceNo"].isin(top_20_transactions))
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

In [139]:
# Create the "basket"

basket = (
    data.groupby(["InvoiceNo", "StockCode"])["Quantity"]
    .sum()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index("InvoiceNo")
)


In [144]:
#Check to make sure you did it right
basket[basket[10002] > 0]

StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,POST,SP1002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_70,gift_0001_80,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
490063,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490136,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490140,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490144,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490229,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547223,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547729,6.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548714,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

In [147]:
basket = basket.map(lambda x: 1 if x > 0 else 0)
basket.head()

StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,POST,SP1002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_70,gift_0001_80,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [173]:
frequent_itemsets = apriori(basket, min_support=0.07, use_colnames=True)
frequent_itemsets

/Users/matehal/Documents/mhaligowski/uw/.venv/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.084615,(20724)
1,0.122253,(20725)
2,0.072927,(20726)
3,0.095688,(20727)
4,0.081539,(20728)
5,0.078128,(20914)
6,0.083496,(21080)
7,0.151558,(21212)
8,0.076282,(21213)
9,0.073094,(21231)


Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

In [174]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(21931),(85099B),0.109558,0.207874,0.071528,0.652884,3.140764,0.048754,2.282021,0.765469
1,(85099B),(21931),0.207874,0.109558,0.071528,0.344095,3.140764,0.048754,1.357578,0.860477
2,(22386),(85099B),0.118394,0.207874,0.075387,0.636750,3.063150,0.050776,2.180663,0.763990
3,(85099B),(22386),0.207874,0.118394,0.075387,0.362658,3.063150,0.050776,1.383255,0.850293


Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.

In [ ]:
I don't see too many opportunities, based on the top 20 products in the UK. Looks like it would be a good idea to promote
product 85099B, since it has hire antecendent support.

Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

In [ ]:
I didn't expect to see so many operations that needed to be performed on the data, before actually applying the apriori algorithm. I was
a little confused with the one hot encoding method, since we had been using Pandas solution for a while, and apparently mlxtend also has 
a function for it.